In [9]:
import requests
import json
import pandas as pd
from models import *

In [10]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
engine = sqlalchemy.create_engine('sqlite:///adverse--events.db', echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2018-11-14 15:05:00,878 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-11-14 15:05:00,880 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,883 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-11-14 15:05:00,884 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,888 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("events")
2018-11-14 15:05:00,889 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,892 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("brands")
2018-11-14 15:05:00,893 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,895 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("brands_events")
2018-11-14 15:05:00,896 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,897 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reactions")
2018-11-14 15:05:00,898 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 15:05:00,900 INFO sql

In [13]:
class Results: 
    
    def get_results(date,events):
        data = []
        for i in range(0, events, 100):
            url = 'https://api.fda.gov/drug/event.json?api_key=iQLpf10vDEd4qDbJfIXK9Tm9SYFXo2cCoy4Ghv62&search=receivedate:' + str(date) + '&limit=100'+'&skip='+str(i)
            r = requests.get(url)
            jsoning_data = r.json()
            data += list(jsoning_data['results'])
        return data
    

In [14]:
Results.get_results(20170101,400)

KeyError: 'results'

In [4]:
class Instantiating_Holidays: 
    
    def holidays(event): #change to holiday_json later
            if event['receiptdate'] == '20171231':
                holiday = 'New Years Eve'
            elif event['receiptdate'] == '20171031':
                 holiday = 'Halloween'
            elif event['receiptdate'] == '20170214':
                holiday = 'Valentine\'s Day'
            elif event['receiptdate'] == '20170228':
                holiday = 'Mardi Gras'
            elif event['receiptdate'] == '20170317':
                holiday = 'St. Patrick\'s Day'
            elif event['receiptdate'] == '20170420':
                holiday = 'Cannabis Day'
            elif event['receiptdate'] == '20170505':
                holiday = 'Cinco de Mayo'
            elif event['receiptdate'] == '20170704':
                holiday = 'Independence Day'
            elif event['receiptdate'] == '20171123':
                holiday = 'Thanksgiving'
            elif event['receiptdate'] == '20171225':
                holiday = 'Christmas'
                
            holiday_name = Holidays(name=holiday,date=str('receiptdate'))
            return holiday_name

In [5]:
## GLOBAL FUNCTIONS

def sex(event):
    patients = []
    sex = []
    if 'patient' in event:
        sex.append(event['patient'])
    else: 
        sex.append('None')
    for i in sex:
        if 'patientsex' in i: 
            patients.append(int(i['patientsex']))
        else: 
            patients.append("None")
    return patients[0]

    
def age(event):
    patients = []
    ages = []
    if 'patient' in event:
        ages.append(event['patient'])
    else: 
        ages.append('None')
    for i in ages:
        if 'patientonsetage' in i: 
            patients.append(int(i['patientonsetage']))
        else: 
            patients.append('None')
    return patients[0]

In [6]:
class Instantiating_Events:

    def event_object(date,events):
        holiday_json = Results.get_results(date,events)
        our_holiday = Instantiating_Holidays.holidays(holiday_json[0])

        for event in holiday_json:
            sex_func = sex(event) 
            age_func = age(event)
            new_event = Adverse_Events(sex=sex_func, age=age_func, holidays = our_holiday)
            
            
            b = []
            if 'openfda' in event['patient']['drug'][0]:
                    if 'brand_name' in event['patient']['drug'][0]['openfda']:
                        b.extend(event['patient']['drug'][0]['openfda']['brand_name'])
                    else:
                        b.extend('None')
                        
            brand_list = []
            for brand in b: 
                brand_result = session.query(Brands).filter(Brands.name == brand).first()
                if brand_result: 
                    brand_list.append(brand_result)
                else: 
                    brand_list.append(Brands(name=brand))
            new_event.brands = brand_list
            
            
            
        
            reactions =  event['patient']['reaction']
            list_of_reactions = []
            for reaction in reactions: 
                list_of_reactions.append(reaction['reactionmeddrapt'])
            
            reaction_list = []
            for reaction in list_of_reactions: 
                reaction_result = session.query(Reactions).filter(Reactions.name == reaction).first()
                if reaction_result: 
                    reaction_list.append(reaction_result)
                else: 
                    reaction_list.append(Reactions(name=reaction))
            new_event.reactions = list(set(reaction_list))
            
            
            session.add(new_event)
            session.commit()

In [7]:
Instantiating_Events.event_object(20171225,1000)

2018-11-14 11:29:14,247 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 11:29:14,248 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,249 INFO sqlalchemy.engine.base.Engine ('PAROXETINE', 1, 0)
2018-11-14 11:29:14,253 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,254 INFO sqlalchemy.engine.base.Engine ('BRISDELLE', 1, 0)
2018-11-14 11:29:14,258 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,259 INFO sqlalchemy.engine.base.Engine ('Paraesthesia', 1, 0)
2018-11-14 11:29:14,263 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE 

2018-11-14 11:29:14,617 INFO sqlalchemy.engine.base.Engine ('Urinary tract infection', 1, 0)
2018-11-14 11:29:14,628 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,633 INFO sqlalchemy.engine.base.Engine ('Pain in extremity', 1, 0)
2018-11-14 11:29:14,637 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,639 INFO sqlalchemy.engine.base.Engine ('Treatment failure', 1, 0)
2018-11-14 11:29:14,641 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,643 INFO sqlalchemy.engine.base.Engine ('Headache', 1, 0)
2018-11-14 11:29:14,648 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reacti

2018-11-14 11:29:14,818 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 11:29:14,820 INFO sqlalchemy.engine.base.Engine (2, 56, 1)
2018-11-14 11:29:14,823 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:14,825 INFO sqlalchemy.engine.base.Engine ('LEVOXYL', 1, 0)
2018-11-14 11:29:14,827 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 11:29:14,830 INFO sqlalchemy.engine.base.Engine (3,)
2018-11-14 11:29:14,834 INFO sqlalchemy.engine.base.Engine INSERT INTO brands (name) VALUES (?)
2018-11-14 11:29:14,837 INFO sqlalchemy.engine.base.Engine ('LEVOXYL',)
2018-11-14 11:29:14,841 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (

2018-11-14 11:29:15,083 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 11:29:15,085 INFO sqlalchemy.engine.base.Engine ('None', 'None', 1)
2018-11-14 11:29:15,090 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:15,092 INFO sqlalchemy.engine.base.Engine ('ENTRESTO', 1, 0)
2018-11-14 11:29:15,094 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 11:29:15,096 INFO sqlalchemy.engine.base.Engine (6,)
2018-11-14 11:29:15,100 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 11:29:15,102 INFO sqlalchemy.engine.base.Engine (6, 6)
2018-11-14 11:29:15,105 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reac

2018-11-14 11:29:15,284 INFO sqlalchemy.engine.base.Engine ('Generalised tonic-clonic seizure', 1, 0)
2018-11-14 11:29:15,287 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:29:15,289 INFO sqlalchemy.engine.base.Engine ('Psoriatic arthropathy', 1, 0)
2018-11-14 11:29:15,293 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions, reactions_events 
WHERE ? = reactions_events.event_id AND reactions.id = reactions_events.reactions_id
2018-11-14 11:29:15,295 INFO sqlalchemy.engine.base.Engine (8,)
2018-11-14 11:29:15,299 INFO sqlalchemy.engine.base.Engine INSERT INTO reactions (name) VALUES (?)
2018-11-14 11:29:15,300 INFO sqlalchemy.engine.base.Engine ('Generalised tonic-clonic seizure',)
2018-11-14 11:29:15,302 INFO sqlalchemy.engine.base.Engine INSERT INTO reactions (name) VALUES (?)
2018-11-14

...

2018-11-14 11:30:43,432 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:43,433 INFO sqlalchemy.engine.base.Engine ('ANTI-BACTERIAL HAND AMERICAN PEACE SIGN', 1, 0)
2018-11-14 11:30:43,435 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:43,435 INFO sqlalchemy.engine.base.Engine ('INSTANT HAND SANITIZER CITRUS SCENT', 1, 0)
2018-11-14 11:30:43,437 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:43,437 INFO sqlalchemy.engine.base.Engine ('RUE 21 DREAM DANCE LOVE VANILLA SPICE ANTI BACTERIAL HAND SANITIZER', 1, 0)
2018-11-14 11:30:43,439 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE

2018-11-14 11:30:43,854 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 11:30:43,856 INFO sqlalchemy.engine.base.Engine (1,)
2018-11-14 11:30:43,858 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 11:30:43,860 INFO sqlalchemy.engine.base.Engine (2, 42, 1)
2018-11-14 11:30:43,865 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:43,867 INFO sqlalchemy.engine.base.Engine ('Status epilepticus', 1, 0)
2018-11-14 11:30:43,869 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:43,871 INFO sqlalchemy.engine.base.Engine ('Measles', 1, 0)
2018-11-14

2018-11-14 11:30:44,048 INFO sqlalchemy.engine.base.Engine ('COPAXONE', 1, 0)
2018-11-14 11:30:44,052 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 11:30:44,054 INFO sqlalchemy.engine.base.Engine (889,)
2018-11-14 11:30:44,060 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 11:30:44,061 INFO sqlalchemy.engine.base.Engine (1743, 889)
2018-11-14 11:30:44,066 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 11:30:44,070 INFO sqlalchemy.engine.base.Engine ('Hypersensitivity', 1, 0)
2018-11-14 11:30:44,074 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions, reactions_events 
WHERE ? 

2018-11-14 11:30:44,276 INFO sqlalchemy.engine.base.Engine (892,)
2018-11-14 11:30:44,280 INFO sqlalchemy.engine.base.Engine INSERT INTO reactions_events (reactions_id, event_id) VALUES (?, ?)
2018-11-14 11:30:44,282 INFO sqlalchemy.engine.base.Engine (215, 892)
2018-11-14 11:30:44,286 INFO sqlalchemy.engine.base.Engine COMMIT
2018-11-14 11:30:44,292 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 11:30:44,295 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 11:30:44,298 INFO sqlalchemy.engine.base.Engine (1,)
2018-11-14 11:30:44,303 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 11:30:44,305 INFO sqlalchemy.engine.base.Engine (1, 58, 1)
2018-11-14 11:30:44,308 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name

2018-11-14 11:30:44,484 INFO sqlalchemy.engine.base.Engine ('Headache', 1, 0)
2018-11-14 11:30:44,488 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions, reactions_events 
WHERE ? = reactions_events.event_id AND reactions.id = reactions_events.reactions_id
2018-11-14 11:30:44,490 INFO sqlalchemy.engine.base.Engine (895,)
2018-11-14 11:30:44,496 INFO sqlalchemy.engine.base.Engine INSERT INTO reactions_events (reactions_id, event_id) VALUES (?, ?)
2018-11-14 11:30:44,500 INFO sqlalchemy.engine.base.Engine ((134, 895), (103, 895), (11, 895))
2018-11-14 11:30:44,504 INFO sqlalchemy.engine.base.Engine COMMIT
2018-11-14 11:30:44,523 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 11:30:44,525 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 11:30:44,527 INFO sqlalchemy.engine.b